# FULL FEATURE

Quick experiment to test if both text and numeric features (full feature) can distinguish AI vs Human writing

**Result**: ~60.2% with full features vs 55% accuracy with numeric-only feature approach

**Conclusion**: The full features approach provides meaningful improvement over numeric-only feature approach

*Note: This was an exploratory experiment. See main.py for the complete, production-ready pipeline.*

In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Cell 2: Load data
df = pd.read_csv('../data/ai_human_content_detection_dataset.csv')

In [ ]:
# Variables for results and evaluation
results={}

highest_score={
       'model': None,
       'score': 0
}

In [ ]:
# Models 
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=100000),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier()
}

In [ ]:
# X data with raw numeric-only values
X_numeric=df[['word_count', 'character_count',
       'sentence_count', 'lexical_diversity', 'avg_sentence_length',
       'avg_word_length', 'punctuation_ratio', 'flesch_reading_ease',
       'gunning_fog_index', 'grammar_errors', 'passive_voice_ratio',
       'predictability_score', 'burstiness', 'sentiment_score']]

In [ ]:
# Vectorised text_content values (essay/assignment free text) with modification
vectorizer = TfidfVectorizer(  
    max_features=950,  
    ngram_range=(1, 3),  # Include bigrams and trigrams
    min_df=2,  # Ignore very rare words
    max_df=0.9,  # Ignore very common words
    sublinear_tf=True,  # Apply sublinear scaling
    use_idf=True,
    smooth_idf=True
    )  
X_text = vectorizer.fit_transform(df['text_content'])  

In [ ]:
# Encoding simple one-worded content_type data to numeric values
X_cat = pd.get_dummies(df['content_type'], drop_first=True)

In [ ]:
# Combining all numeric and encoded numeric values in one sparse array
X_non_text = np.hstack([X_numeric.values, X_cat.values])
X_full = hstack([X_non_text, X_text])

In [ ]:
# Y value
y=df['label']

In [ ]:
# Train/Test data
X_train, X_test, y_train, y_test = train_test_split(
       X_full,
       y,
       test_size=0.2,
       random_state=42,
       stratify=y
)

In [ ]:
# Impute nan/missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [ ]:
# Train and evaluate all models (no scaling or PCA)
for name, model in models.items():
    model.fit(X_train_imputed, y_train)
    y_pred = model.predict(X_test_imputed)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"RAW MODEL EVAL: {name}: {accuracy:.3f}")
best_model = max(results, key=results.get)

# Store highest score
if results[best_model] > highest_score["score"]:
       highest_score["score"] = results[best_model]
       highest_score["model"] = best_model

In [ ]:
# Scale training and test data
scaler = StandardScaler(with_mean=False)  # with_mean=False for sparse matrices
X_train_scaled = scaler.fit_transform(X_train_imputed) #fit and transform only on training data
X_test_scaled= scaler.transform(X_test_imputed)

In [ ]:
# Convert sparse matrices to dense (FOR PCA)
X_train_scaled_dense = X_train_scaled.toarray()  
X_test_scaled_dense = X_test_scaled.toarray()

In [ ]:
# Train and evaluate all models with scaled data
for name, model in models.items():
    model.fit(X_train_scaled_dense, y_train)
    y_pred = model.predict(X_test_scaled_dense)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"SCALED MODEL EVAL (DENSE): {name}: {accuracy:.3f}")
best_model = max(results, key=results.get)

# Store highest score
if results[best_model] > highest_score["score"]:
       highest_score["score"] = results[best_model]
       highest_score["model"] = best_model

In [ ]:
# Apply PCA to capture 95% variance
pca=PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled_dense)
X_test_pca = pca.transform(X_test_scaled_dense)

In [ ]:
# Train and evaluate all models with scaled + PCA data
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"PCA APPLIED MODEL EVAL {name}: {accuracy:.3f}")
    
# Store highest score
if results[best_model] > highest_score["score"]:
       highest_score["score"] = results[best_model]
       highest_score["model"] = best_model
best_model = max(results, key=results.get)

In [ ]:
# === EXPERIMENT: Optimizing LogisticRegression C parameter ===
# After finding LogisticRegression was initially best, tested different regularization strengths
# RESULT: Default C=1.0 was optimal for 100 features, but with 600 features, C=0.1 performed better
# This led to discovering that more text features (600 vs 100) was more impactful than C tuning
lr_models = {
    'LR_C=0.1': LogisticRegression(C=0.1, random_state=42, max_iter=10000),
    'LR_C=1.0': LogisticRegression(C=1.0, random_state=42, max_iter=10000), 
    'LR_C=10.0': LogisticRegression(C=10.0, random_state=42, max_iter=10000),
}

for name, model in lr_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print(f"Best RAW Model: {best_model} with accuracy: {results[best_model]:.3f}")

In [ ]:
# THOUGHT PROCESS AND EVAL:
# Grid search with cross-validation and multiple C values worsened score and same with linearSVC
# optimised second best model (lr) with best .54 lower than svm
# optimised nbm model scored second best optimised at 5.8

In [ ]:
# Final Model
best_svm = SVC(C=1, gamma='scale', kernel='rbf')
best_svm.fit(X_train_pca, y_train)
y_pred_final = best_svm.predict(X_test_pca)

In [ ]:
# Performance
print("Final Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_final):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_final))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_final))

In [ ]:
#TESTING WITH NMB MODEL

In [ ]:
# NMB model fail
nb = MultinomialNB()

# Make all values positive by adding the minimum
X_train_positive = X_train_imputed.toarray() - X_train_imputed.toarray().min()
X_test_positive = X_test_imputed.toarray() - X_test_imputed.toarray().min()

In [ ]:
nb.fit(X_train_positive, y_train)  
y_pred_nmb = nb.predict(X_test_positive)
accuracy = accuracy_score(y_test, y_pred_nmb)
print(f"NBM RAW: {accuracy}")

In [ ]:
# This ensures values stay between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_positive)
X_test_scaled = scaler.transform(X_test_positive)

In [ ]:
nb.fit(X_train_scaled, y_train)
y_pred_scaled= nb.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred_scaled)
print(f"NBM SCALED: {accuracy}")

In [ ]:
# PCA
pca=PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
# After PCA, shift to positive
X_train_pca_positive = X_train_pca - X_train_pca.min()
X_test_pca_positive = X_test_pca - X_test_pca.min()

In [ ]:
nb.fit(X_train_pca_positive, y_train)
y_pred_pca = nb.predict(X_test_pca_positive)
accuracy = accuracy_score(y_test, y_pred_pca)
print(f"NBM PCA: {accuracy}")